In [33]:
%matplotlib notebook

In [1]:
import os, sys

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import multiprocessing as mp

import elephant
from neo.core import AnalogSignal
import quantities as pq
import xarray as xr

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

In [2]:
#exp_data_dir = '/Users/atleeskelandrimehaug/Dropbox (Simula)/Data/Allen_data/manually_downloaded_sessions/ecephys_cache_dir_10_31'

exp_lfp_probes_data_dir = '/Volumes/My Passport/ecephys_cache_dir_10_31'

manifest_path = os.path.join(exp_lfp_probes_data_dir, "manifest.json")
#manifest_path = os.path.join(exp_data_dir, "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
def read_in_lfp(session, session_id, structure='VISp', stim='flashes', color=1., trial_window_start = -0.5, trial_window_end = 0.1):
    for probe_id in session.probes.index.values:
        mask = session.channels.probe_id==probe_id
        ecephys_structures = session.channels[mask].ecephys_structure_acronym.unique()
        
        print(ecephys_structures)
        
        if structure in ecephys_structures:
            print(session_id, session.probes.loc[probe_id].description, probe_id, )
            
            lfp = session.get_lfp(probe_id)
            mask = (session.stimulus_presentations.stimulus_name==stim) & \
                   (session.stimulus_presentations.color==color)
            presentation_table = session.stimulus_presentations[mask]
            presentation_times = presentation_table.start_time.values
            presentation_ids = presentation_table.index.values
            
            mask = (session.channels.ecephys_structure_acronym==structure) & \
                   (session.channels.ecephys_structure_acronym.isin([structure]))
            channel_ids = session.channels[mask].index.values
            channel_vertical_position = session.channels.probe_vertical_position[channel_ids]
            
            sf = np.round(probes.lfp_sampling_rate[probe_id])# / 2.5
            trial_window = np.arange(trial_window_start, trial_window_end, 1/sf)
            time_selection = np.concatenate([trial_window + t for t in presentation_times])

            inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                               names=('presentation_id', 'time_from_presentation_onset'))

            ds = lfp.sel(time=time_selection, method='nearest').to_dataset(name='aligned_lfp')
            ds = ds.assign(time=inds).unstack('time')
            aligned_lfp = ds['aligned_lfp']
            
            lfp = aligned_lfp.sel(channel=slice(np.min(channel_ids), np.max(channel_ids)))
            mean_lfp = lfp.mean('presentation_id')
            channel_vertical_pos = session.channels.probe_vertical_position[np.array(lfp.channel)]
            
            s = dict()
            s['lfp_mean'] = mean_lfp
            s['lfp'] = lfp
            s['channel_vertical_pos'] = channel_vertical_pos
            s['time'] = trial_window
            s['sf'] = sf
            return s

In [21]:
session_idx = 0
list_session_ids = [sessions.index.values[session_idx]]
list_session_ids

[847657808]

In [22]:
trial_window_start = -0.5
trial_window_end = 1.0

In [23]:
X = dict()
stim = 'flashes'
color = +1.



#for i, session_id in enumerate(sessions.index.values[:]):
for i, session_id in enumerate(list_session_ids):

    try:
        X[session_id] = dict()
        session = cache.get_session_data(session_id)
        X[session_id]['cortex'] = read_in_lfp(session, session_id, structure='VISp', stim=stim, color=color, trial_window_start = trial_window_start, trial_window_end = trial_window_end)
        #X[session_id]['lgn'] = read_in_lfp(session, session_id, structure='LGd', stim=stim, color=color, trial_window_start = trial_window_start, trial_window_end = trial_window_end)
        X[session_id]['name'] = session_id
        
        name = str(session_id)+'_lfp.npy'
        path = os.path.join('..', '..', 'processed_data', 'lfp', 'flash_white', 'trial_window_'\
             +str(trial_window_start).replace('.','pnt')+\
             '_to_'+str(trial_window_end).replace('.','pnt')+'s', name)
        np.save(path, X[session_id])
        
    except:
        print('==========================')
        print(session_id)
        print('==========================')
        pass

#np.save('../processed_data/animals_white_flash_LFP.npy', X)  

['APN' 'NOT' 'DG' 'HPF' 'ProS' 'CA1' 'VISam' nan]
['APN' 'MB' 'NOT' 'DG' 'ProS' 'VISpm' nan]
['DG' 'POST' 'SUB' 'VISp' nan]
847657808 probeC 848037572


In [24]:
X = dict()
stim = 'flashes'
color = -1.

#for i, session_id in enumerate(sessions.index.values[:]):
for i, session_id in enumerate(list_session_ids):

    try:
        X[session_id] = dict()
        session = cache.get_session_data(session_id)
        X[session_id]['cortex'] = read_in_lfp(session, session_id, structure='VISp', stim=stim, color=color, trial_window_start = trial_window_start, trial_window_end = trial_window_end)
        #X[session_id]['lgn'] = read_in_lfp(session, session_id, structure='LGd', stim=stim, color=color, trial_window_start = trial_window_start, trial_window_end = trial_window_end)
        X[session_id]['name'] = session_id
        
        name = str(session_id)+'_lfp.npy'
        path = os.path.join('..', '..', 'processed_data', 'lfp', 'flash_black', 'trial_window_'\
             +str(trial_window_start).replace('.','pnt')+\
             '_to_'+str(trial_window_end).replace('.','pnt')+'s', name)
        np.save(path, X[session_id])
        
    except:
        print('==========================')
        print(session_id)
        print('==========================')
        pass


['APN' 'NOT' 'DG' 'HPF' 'ProS' 'CA1' 'VISam' nan]
['APN' 'MB' 'NOT' 'DG' 'ProS' 'VISpm' nan]
['DG' 'POST' 'SUB' 'VISp' nan]
847657808 probeC 848037572
